# Dealing with Data Spring 2020 – Class 6

---

# Connect to our db: 

In [1]:
import sqlite3
import pandas as pd

In [2]:
!pwd

/Users/siegmanA/Desktop/NYU-Dealing-With-Data-Spring-2020/Class 6


In [3]:
con = sqlite3.connect('/Users/siegmanA/Desktop/NYU-Dealing-With-Data-Spring-2020/Class 5/class_practice.db') # this is how we are going to create our database, 
                                         # calling it 'citibikeData.db'

# "con" stands for "connection" – this is telling SQLite what database to use

---

# GROUP BY

```sql
SELECT A1, Aggregation Function [count(*), sum(*), avg(*), min, etc.]
FROM T1, T2, ... Tm
WHERE condition
GROUP BY A1
```

`count(*)` counts the number of rows in the group <br> 
`count(attr)` counts the number of rows in the group with non-null values for the attribute <br> 
`count(DISTINCT attr)` counts the number of distinct, non-null values for the attributes in the group <br> 
`max(attr)` row with maximum attribute value in the group <br>
`min(attr)` row with minimum value in the group <br>
`sum(attr)` sum values of selected rows in the group <br>
`avg(attr)` estimates the average attribute in the group

---

# Exercise 1: 

> What borough has the lowest average travel time from our set?

In [4]:
# your code here

# Solution

In [5]:
check = pd.read_sql("""SELECT borough, ROUND(AVG(travel_time),0) as average_travel_time
                       FROM TrafficData 
                       GROUP BY borough
                       ORDER BY average_travel_time ASC
                       """
                    , con=con)
check

,borough,average_travel_time
0,Staten island,54.0
1,Staten Island,120.0
2,Brooklyn,204.0
3,Queens,217.0
4,Manhattan,219.0
5,Bronx,220.0


---

# Exercise 2: 

>  What 5 CitiBike stations (by stationName) have the most docks on average across our set? 

In [6]:
# your code here

# Solution

In [7]:
check = pd.read_sql("""SELECT stationName, ROUND(AVG(availableDocks),0) as average_available_docks
                       FROM StationsData
                       GROUP BY stationName
                       ORDER BY average_available_docks DESC
                       LIMIT 5;
                       """, 
                       con=con)
check

,stationName,average_available_docks
0,E 10 St & Avenue A,78.0
1,FDR Drive & E 35 St,68.0
2,Pershing Square South,63.0
3,Division Ave & Hooper St,61.0
4,W 84 St & Broadway,61.0


---

# HAVING

```sql 
SELECT A1, Aggregation Function
FROM T1, T2, ... Tm
WHERE condition
GROUP BY A1
HAVING Aggregation Function Condition
```
<br>

`WHERE` applies to rows _before_ computing the aggregate <br>
`HAVING` applies to the aggregate value only

In [8]:
# WHERE

check = pd.read_sql("""SELECT borough, ROUND(AVG(travel_time),0) as average_travel_time
                       FROM TrafficData 
                       WHERE travel_time > 100
                       GROUP BY borough
                       """
                    , con=con)
check

,borough,average_travel_time
0,Bronx,308.0
1,Brooklyn,376.0
2,Manhattan,280.0
3,Queens,253.0
4,Staten Island,200.0


In [9]:
# HAVING 

In [10]:
check = pd.read_sql("""SELECT borough, ROUND(AVG(travel_time),0) as average_travel_time
                       FROM TrafficData 
                       GROUP BY borough
                       HAVING average_travel_time > 100
                       """
                    , con=con)
check

,borough,average_travel_time
0,Bronx,220.0
1,Brooklyn,204.0
2,Manhattan,219.0
3,Queens,217.0
4,Staten Island,120.0


---

# Exercise 3: 

> Return the names of any CitiBike stations with greater than 50 average available docks.

In [11]:
# your code here

# Solution

In [12]:
check = pd.read_sql("""SELECT stationName, ROUND(AVG(availableDocks),0) as average_available_docks
                       FROM StationsData
                       GROUP BY stationName
                       HAVING average_available_docks > 50
                       """, 
                       con=con)
check

,stationName,average_available_docks
0,1 Ave & E 18 St,51.0
1,6 Ave & W 34 St,57.0
2,8 Ave & W 33 St,56.0
3,9 Ave & W 22 St,51.0
4,Central Park W & W 96 St,52.0
5,Division Ave & Hooper St,61.0
6,E 10 St & Avenue A,78.0
7,E 2 St & Avenue B,56.0
8,E 2 St & Avenue C,53.0
9,E 6 St & Avenue B,53.0


---

# Conditional Construction: CASE

```sql
CASE
    WHEN condition THEN result
    WHEN condition2 THEN result2
    ELSE result
END
```

---

# Exercise 4: 

> Map each CitiBike station to either a ("Fewer than 30 available docks", "Between 30 and 60 available docks", or "Greater than 60 available docks") to a "availability" bucket

In [13]:
# your code here

# Solution

In [14]:
check = pd.read_sql("""SELECT stationName,
                       CASE 
                           WHEN ROUND(AVG(availableDocks),0) < 30 THEN "fewer than 30 available docks" 
                           WHEN 30 <= ROUND(AVG(availableDocks),0) < 60 THEN "between 30 and 60 available docks"
                           ELSE "greater than 60 available docks"
                        END AS availability_bucket
                       FROM StationsData
                       GROUP BY stationName
                       """, 
                       con=con)
check

,stationName,availability_bucket
0,1 Ave & E 110 St,fewer than 30 available docks
1,1 Ave & E 16 St,between 30 and 60 available docks
2,1 Ave & E 18 St,between 30 and 60 available docks
3,1 Ave & E 30 St,fewer than 30 available docks
4,1 Ave & E 44 St,fewer than 30 available docks
...,...,...
930,Wyckoff Av & Stanhope St,fewer than 30 available docks
931,Wyckoff St & Bond St,fewer than 30 available docks
932,Wyckoff St & Nevins St,fewer than 30 available docks
933,Wythe Ave & Metropolitan Ave,fewer than 30 available docks


---

# Exercise 5: 

> Bucket each Borough in our travel data into "Slow", "OK", and "Heavy Traffic" buckets according to your own definitions of the three. 

In [15]:
# your code here

# Solution

In [16]:
check = pd.read_sql("""SELECT borough, ROUND(AVG(travel_time),0) as average_travel_time
                       FROM TrafficData 
                       GROUP BY borough
                       """
                    , con=con)
check

,borough,average_travel_time
0,Bronx,220.0
1,Brooklyn,204.0
2,Manhattan,219.0
3,Queens,217.0
4,Staten Island,120.0
5,Staten island,54.0


In [17]:
check = pd.read_sql("""SELECT borough, 
                       CASE 
                           WHEN ROUND(AVG(travel_time),0) > 210 THEN "SLOW"
                           WHEN 200 < ROUND(AVG(travel_time),0) <= 210 THEN "OK"
                           ELSE "FAST"
                       END AS speed
                       FROM TrafficData 
                       GROUP BY borough
                       """
                    , con=con)
check

,borough,speed
0,Bronx,SLOW
1,Brooklyn,OK
2,Manhattan,SLOW
3,Queens,SLOW
4,Staten Island,OK
5,Staten island,OK


---

# SQL JOINS

First and foremost, any join requires a common data field found in both tables that enables the combination. However, those data fields don't need to share a common name, but they must be the same data type!

NB: LEFT and RIGHT are determined by order after the FROM command! 

<img src="https://i.stack.imgur.com/VQ5XP.png">

# INNER JOIN

> This is the default SQL JOIN, thus you don't even need to specify "INNER". <br> <br>

1) A new table is created with the columns of both tables you're trying to combine, <br><br>
2) It then looks to match values between the columns you specify in your 'ON' statement, <br><br>
3) SQL will start with the first value of the specified column in the first table, then look through every value in the specified column of the second table for a match, <br><br>
4) If there is a match, SQL will copy the data from both the row of the first table and the row of the second table and put it in the newly created table; It won't add any rows that didn't have a match

<br>

`AKA an INNER JOIN is going to return all rows when the comparisoin fields (in the "ON" statement") are matching in BOTH tables.`

In [18]:
from IPython.display import HTML
HTML('<img src="https://dataschool.com/assets/images/how-to-teach-people-sql/innerJoin/innerJoin_3.gif">')

# LEFT JOIN

> LEFT refers to the first table, or the table you'll be joining to. <br> <br>
> For any rows in the LEFT table that don't have a match in the RIGHT table, it still adds that row to the next table but puts nulls for the missing columns

<br> 

`AKA a LEFT JOIN is going to return all rows from the LEFT table with matching rows from the RIGHT table`

In [19]:
from IPython.display import HTML
HTML('<img src="https://dataschool.com/assets/images/how-to-teach-people-sql/leftJoin/leftJoin_1.gif">')

# RIGHT JOIN

> RIGHT refers to the second table, or the table you'll be joining in. <br> <br>
> A RIGHT JOIN can be re-written as a LEFT JOIN, and is thus much more rare in practice.

# FULL OUTER JOIN

> FULL OUTER refers to the first table, or the table you'll be joining to (in this case, the Facebook table since it comes before LinkedIn in the query). <br> 
> After completing a LEFT join of the data, it basically perorms a RIGHT join – only checking ot see if each value is present in the joined table. If it is not, SQL will add this row to the new table and put nulls for the columns from the other table.

In [20]:
from IPython.display import HTML
HTML('<img src="https://dataschool.com/assets/images/how-to-teach-people-sql/fullOuter/fullOuter_1.gif">')

# UNION

In [21]:
# https://dataschool.com/how-to-teach-people-sql/union-animated/

In [22]:
from IPython.display import HTML
HTML('<img src="https://dataschool.com/assets/images/how-to-teach-people-sql/union/union_2.gif">')

# CROSS JOIN

In [23]:
# https://dataschool.com/how-to-teach-people-sql/cross-join-animated/

In [24]:
from IPython.display import HTML
HTML('<img src="https://dataschool.com/assets/images/how-to-teach-people-sql/crossJoin/crossJoin_1.gif">')

In [25]:
# PRACTICE QUERIES FOR EACH, MAKE THEM GUESS WHICH TO USE AND THEN USE IT

---

# Subqueries

Subqueries are temporary tables created with nested SELECT statements where a table should be that allow for deeper analysis with SQL. 

In [26]:
# AS 

In [ ]:
# WITH clauses create a Common Table Expression (CTE) and enables sub-query empowered 
# analyses with easier SQL codebase

In [ ]:
# EXAMPLES

---